# Moduł 8.3 Wyszukiwanie semantyczne
## Indeksowanie i przeszukiwanie tekstu w oparciu o międzyjęzykowe reprezentacje dystrybucyjne



In [1]:
# Instalacja laserembeddings 
!pip install laserembeddings # https://pypi.org/project/laserembeddings/
from laserembeddings import Laser
!python -m laserembeddings download-models

     |████████████████████████████████| 108.8 MB 1.0 MB/s eta 0:00:01
  Using cached transliterate-1.10.2-py2.py3-none-any.whl (45 kB)
Processing /Users/ppez/Library/Caches/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94/sacremoses-0.0.35-cp37-none-any.whl
  Using cached subword_nmt-0.3.7-py2.py3-none-any.whl (26 kB)
You should consider upgrading via the '/Users/ppez/anaconda3/bin/python -m pip install --upgrade pip' command.

✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [2]:
# Międzyjęzykowe reprezentacje zdań

from laserembeddings import Laser
laser = Laser()

test_embeddings_en = laser.embed_sentences(
    ['Two dogs and a cat.', # 1A
     'Hello, how are you?'], # 1B
    lang='en')  # parametr lang jest używany do segmentacji

test_embeddings_pl = laser.embed_sentences(
    ['Dwa psy i kot.', # 2A Podobieństwo do 'Hello, how are you?' nawet przy nieoczywistej synonimii
     'Hejka, co słychać?'], # 2B
    lang='pl')

print(test_embeddings_en)
print(test_embeddings_pl)

print(type(test_embeddings_en[0]))
print(test_embeddings_en[0].shape)

[[ 0.00466173  0.00013475  0.00118413 ...  0.00125459  0.00705035
   0.01940012]
 [ 0.00449018  0.00083818  0.00012739 ...  0.00020635  0.01097564
  -0.00251776]]
[[ 5.5652321e-03  1.0571270e-05  1.2139205e-03 ... -6.7800356e-05
  -3.4577381e-03  2.1616975e-02]
 [ 6.5513351e-03  3.1476589e-03 -1.2269928e-03 ...  6.8447008e-03
   1.1222734e-02  3.3674815e-03]]
<class 'numpy.ndarray'>
(1024,)


In [3]:
# Podobieństwo kosinusowe

from sklearn import metrics 
import numpy as np

print("Cosine similarity")
similarity = metrics.pairwise.cosine_similarity(test_embeddings_en,test_embeddings_pl)
print(similarity)

assert(similarity[0][0]>similarity[0][1])
assert(similarity[1][0]<similarity[1][1])

Cosine similarity
[[0.88267004 0.3634114 ]
 [0.3035773  0.7367395 ]]


In [4]:
# Segment tłumaczeniowy z korpusu Paralela (korpus Europarl -- użyty do treningu modelu LASER)

embeddings_pl_en = laser.embed_sentences(
    ['Głosowałam za przyjęciem przedmiotowego dokumentu, którego celem jest wzmocnienie praw pacjentów w transgranicznej opiece zdrowotnej.',
     'I voted in favour of this document, which aims to strengthen patients rights in cross-border healthcare.'
     ],
    lang=['pl','en'])

print(metrics.pairwise.cosine_similarity(embeddings_pl_en))


[[0.99999994 0.9804411 ]
 [0.9804411  1.0000002 ]]


In [ ]:
# Bardziej realistyczny przykład

# Tytuł podobnej informacji prasowej
embeddings_pl_en = laser.embed_sentences(
    ['Meghan Markle na wojnie przeciw rzecznikom Pałacu. Nie czuła się przez nich chroniona.', # plotek.pl
     'Meghan Markle felt ‘unprotected’ by royals during stressful pregnancy' # nbcnews.com
     ],
    lang=['pl','en'])

print(metrics.pairwise.cosine_similarity(embeddings_pl_en))



[[1.0000001  0.71394753]
 [0.71394753 1.0000001 ]]


In [ ]:
# Spróbujmy ocenić dokładność (accuracy) tej metody na zbiorze prawdziwych ekwiwalentów z korpusu Paralela
# http://paralela.clarin-pl.eu
# http://paralela.clarin-pl.eu/api/index/sources

import requests

# Pobieramy segmenty tłumaczeniowe wraz z metadanymi
seg_limit=500
cordis_segments = requests.get("http://paralela.clarin-pl.eu/api/index/"+
                "segments?query=source:CORDIS&start=0&rows={}".format(seg_limit))
print("Request status: {}.".format(cordis_segments.status_code))

seg_dict = cordis_segments.json()

print("Retrieved {} segments. ".format(len(seg_dict)))

print(next(iter(seg_dict.values())))


Request status: 200.
Retrieved 500 segments. 
{'text_id_en': '445n', 'authors_pl': [''], 'license_en': 'CC-BY', 'source': 'CORDIS', 'medium': 'kanal_internet', 'title_m_en': 'Community Research and Development Information Service', 'title_a_en': "US researchers develop techniques for 'ethical' stem cell lines", 'lang_trg': 'pol', 'authors_en': [''], 'seg_pl_txt': 'Zespół amerykańskich badaczy uważa, że znalazł sposób na wyprowadzenie linii embrionalnych komórek macierzystych bez konieczności niszczenia embrionu źródłowego.', 'license_pl': 'CC-BY', 'genre': 'typ_publ', 'url_en': 'http://cordis.europa.eu', 'id': 'danXp', 'lang_src': 'eng', 'alignment_type': 'SIMPLE', 'title_m_pl': 'Wspólnotowy Serwis Informacyjny Badań i Rozwoju', 'seq': 1, 'seg_en_txt': 'A team of US researchers believes that it has found a way to develop embryonic stem cell lines without destroying the source embryo.', 'title_a_pl': 'Badacze w USA opracowują metody "etycznego" wyprowadzania linii komórek macierzystych'

In [ ]:
# Ograniczmy się do unikalnych tytułów artykułów

title_pairs = []
for seg_id in seg_dict:
  title_pair = (seg_dict[seg_id]['title_a_en'],seg_dict[seg_id]['title_a_pl'])
  if title_pair not in title_pairs: 
    title_pairs.append(title_pair)

print(title_pairs[0:5])
print("Unique title pairs: {}".format(len(title_pairs)))


[("US researchers develop techniques for 'ethical' stem cell lines", 'Badacze w USA opracowują metody "etycznego" wyprowadzania linii komórek macierzystych'), ('Homing pigeons smell their way home', 'Gołębie powracające do domu znajdują drogę kierując się węchem'), ('Environmental forensics conference, UK', 'Konferencja poświęcona środowiskowej medycynie sądowej, Wielka Brytania'), ('Noctilucent clouds found on Mars', 'Na Marsie odkryto obłoki srebrzyste'), ("It's raining cosmic dust", 'Deszcz kosmicznego pyłu')]
Unique title pairs: 34


In [ ]:
# Podobieństwo ekwiwalentych tytułów 

sims = []
for ti, title_pair in enumerate(title_pairs):
  en_vec = laser.embed_sentences([title_pair[0]],lang='en')
  pl_vec = laser.embed_sentences([title_pair[1]], lang = 'pl')
  cos_sim = metrics.pairwise.cosine_similarity(en_vec,pl_vec)
  sims.append(cos_sim[0][0])
  print("{}. {}\n{}\n{}".format(ti+1,title_pair,cos_sim, "-"*50))


1. ("US researchers develop techniques for 'ethical' stem cell lines", 'Badacze w USA opracowują metody "etycznego" wyprowadzania linii komórek macierzystych')
[[0.8680676]]
--------------------------------------------------
2. ('Homing pigeons smell their way home', 'Gołębie powracające do domu znajdują drogę kierując się węchem')
[[0.7118055]]
--------------------------------------------------
3. ('Environmental forensics conference, UK', 'Konferencja poświęcona środowiskowej medycynie sądowej, Wielka Brytania')
[[0.85798585]]
--------------------------------------------------
4. ('Noctilucent clouds found on Mars', 'Na Marsie odkryto obłoki srebrzyste')
[[0.704545]]
--------------------------------------------------
5. ("It's raining cosmic dust", 'Deszcz kosmicznego pyłu')
[[0.7869777]]
--------------------------------------------------
6. ('Ocean census hints at immense diversity of marine microbes', 'Badanie oceanów sygnalizuje ogromną różnorodność morskich mikroorganizmów')
[[0.

In [ ]:
print("Average similarity: {}".format(sum(sims)/(len(sims))))


Average similarity: 0.8565035585094901


In [ ]:
# Batch mode
embeddings_en = laser.embed_sentences(
    [t[0] for t in title_pairs],
    lang='en')

embeddings_pl = laser.embed_sentences(
    [t[0] for t in title_pairs],
    lang='pl')

In [ ]:
from sklearn import metrics 
from operator import itemgetter
"""
query: query text
texts: lista tekstów z kolekcji
candidate_vecs: kolekcja wektorów tekstów w których szukamy maks. k najbliższych sąsiadów o min. podobieństwie kosinusowym
laser: instancja LASER-a
lang: język tekstu zapytania
"""
def knn_search(query, candidate_vecs, texts, laser, lang, max_k, min_sim=0.6):
    query_vec = laser.embed_sentences([query],lang=lang)
    res = []
    for vi,v in enumerate(candidate_vecs):
        cos = metrics.pairwise.cosine_similarity([v],query_vec)
        if cos[0][0] >= min_sim:
           res.append((vi,cos[0][0]))
    res = sorted(res, key=itemgetter(1), reverse=True)
    return [[ri, r, texts[r[0]]] for ri,r in enumerate(res[0:max_k])]

In [ ]:
# Liczymy accuracy na 34 parach tłumaczeniowych

hits = 0
misses = 0

for pi, title_pair in enumerate(title_pairs):
  print("\n\n{}. Top equivalents for {}:".format(pi+1, title_pair))
  equivalents = knn_search(title_pair[1], candidate_vecs=embeddings_en, 
                           texts= [t[0] for t in title_pairs], lang='pl', laser=laser, max_k=1, min_sim=0.5)
  print(equivalents)
  if equivalents[0][2]==title_pair[0]:
    hits+=1
  else:
    misses+=1



1. Top equivalents for ("US researchers develop techniques for 'ethical' stem cell lines", 'Badacze w USA opracowują metody "etycznego" wyprowadzania linii komórek macierzystych'):
[[0, (0, 0.8680676), "US researchers develop techniques for 'ethical' stem cell lines"]]


2. Top equivalents for ('Homing pigeons smell their way home', 'Gołębie powracające do domu znajdują drogę kierując się węchem'):
[[0, (1, 0.7118054), 'Homing pigeons smell their way home']]


3. Top equivalents for ('Environmental forensics conference, UK', 'Konferencja poświęcona środowiskowej medycynie sądowej, Wielka Brytania'):
[[0, (2, 0.85798585), 'Environmental forensics conference, UK']]


4. Top equivalents for ('Noctilucent clouds found on Mars', 'Na Marsie odkryto obłoki srebrzyste'):
[[0, (3, 0.704545), 'Noctilucent clouds found on Mars']]


5. Top equivalents for ("It's raining cosmic dust", 'Deszcz kosmicznego pyłu'):
[[0, (4, 0.78697765), "It's raining cosmic dust"]]


6. Top equivalents for ('Ocean c

In [ ]:
print("\n Accuracy: {}/({}+{}) = {}".format(hits,hits,misses,(hits/(hits+misses))))



 Accuracy: 34/(34+0) = 1.0


## Indeskowanie wektorów za pomocą FAISS

In [ ]:
# Instalujemy wersję CPU albo GPU
#!pip install faiss
!pip install faiss-gpu

     |████████████████████████████████| 67.7MB 88kB/s 


In [ ]:
# Budowa indeksu z domyślnymi parametrami
import faiss

def build_faiss(vecs,dim=1024):
  index = faiss.IndexFlatL2(dim) # brute-force L2 distance search
  index.add(np.asarray(vecs))
  print("Total vectors in the index: {}.".format(index.ntotal))
  return index

In [ ]:
index_pl = build_faiss(vecs=embeddings_pl)
index_en = build_faiss(vecs=embeddings_en)


Total vectors in the index: 34.
Total vectors in the index: 34.


In [ ]:
# Przeszukiwanie indeksu FAISS
def faiss_search(query_vec, faiss_index, max_k = 10):
    res = []
    scores, ids = faiss_index.search(np.array([query_vec]), max_k)  # Odległości L2
    for si, id in enumerate(ids[0]):
        res.append((id,scores[0][si]))
    return res

In [ ]:
 #Przykłady odpytywania indeksów

# 1. EN-PL
query_en = laser.embed_sentences(['cosmic dust'], lang='en')[0]
print("Wektor angielskiego zdania o wym. {}: {}".format(query_en.shape, query_en))

res = faiss_search(query_vec=query_en, faiss_index=index_pl, max_k = 3)
print("\nWyniki: {}".format(res))
for r in res:
  print(title_pairs[r[0]])


Wektor angielskiego zdania o wym. (1024,): [ 0.00083235 -0.00024976 -0.00062988 ... -0.0013023   0.00963623
  0.01447   ]

Wyniki: [(4, 0.09741208), (15, 0.24053705), (7, 0.26407197)]
("It's raining cosmic dust", 'Deszcz kosmicznego pyłu')
('Final evaluation of eTEN Programme', 'Końcowa ocena programu eTEN')
('European Venture Market, Potsdam', 'European Venture Market, Poczdam')


In [ ]:
# 2. PL-EN
query_pl = laser.embed_sentences(['końcowa ewaluacja'], lang='pl')[0]
print("Wektor angielskiego zdania o wym. {}: {}".format(query_pl.shape, query_pl))

res = faiss_search(query_vec=query_pl, faiss_index=index_en, max_k = 3)
print("\nWyniki: {}".format(res))
for r in res:
  print(title_pairs[r[0]])


Wektor angielskiego zdania o wym. (1024,): [ 0.00800916 -0.0014777  -0.00325542 ...  0.00224968  0.02335127
  0.01597966]

Wyniki: [(15, 0.14528365), (26, 0.29557374), (20, 0.3106899)]
('Final evaluation of eTEN Programme', 'Końcowa ocena programu eTEN')
('Czech Republic seeks to boost innovation', 'Czechy daza do wprowadzenia innowacji na szersza skale')
('Nanotechnologies for drug delivery', 'Notechnologie w dozowaniu leków')
